# 0) Import, load, etc.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import time
import threading
import datetime
import multiprocessing as mp
from multiprocessing import Process, Manager
from itertools import repeat
import multiprocessing as mp
import os
import pprint
from magictree import * #tree layout
import pickle
import sys

In [ ]:
mimic = sorted(np.load('mimic_sequences.npy'))
p2p_jaccard = np.load('p2p_jaccard.npy')
p2p_wuplus = np.load('p2p_wuplus.npy')

In [ ]:
np.random.seed(100)

# 1) Select datapoint

In [ ]:
#2519

In [ ]:
i2e_index = 300#np.random.choice(len(mimic))
print(i2e_index)
i2e = mimic[i2e_index]
print(i2e)

# 2) Get k REAL neighbours

In [ ]:
k1 = 50

In [ ]:
def get_closest_datapoints_indexes(start_index,dist_matrix,k):
    candidates = sorted(enumerate(dist_matrix[start_index]),key=lambda x:x[1])[:k]
    return list(candidates)

In [ ]:
wup_nbh_indx = [el[0] for el in get_closest_datapoints_indexes(i2e_index,p2p_wuplus,k1)] 
jac_nbh_indx = [el[0] for el in get_closest_datapoints_indexes(i2e_index,p2p_jaccard,k1)] 

# 3) Flatten

In [ ]:
def flat_dp(dp_index):
    codes = list(set([a for b in mimic[dp_index][:-1] for a in b]))
    fdp = {}
    for c in codes:  
        seq = [1 if c in el else 0 for el in mimic[dp_index][:-1][::-1]] #already rev
        decay = [1/2**i for i in range(1,len(mimic[dp_index]))]
        #print(c,seq,decay,np.multiply(seq,decay),sum(np.multiply(seq,decay)))
        fdp[c]=sum(np.multiply(seq,decay))
    return fdp
        
def flat_cloud(kdp):
    local_flat = [flat_dp(n) for n in kdp]
    all_codes = sorted(list(set([a for b in [list(lf.keys()) for lf in local_flat] for a in b])))
    feat_mx = np.full((len(kdp),len(all_codes)),0.)
    for row,lf in enumerate(local_flat):
        for (k,v) in lf.items():
            feat_mx[row][all_codes.index(k)]=v
    return feat_mx,all_codes
    

In [ ]:
wup_nbh_flat,wup_codes = flat_cloud(wup_nbh_indx)
jac_nbh_flat,jac_codes = flat_cloud(jac_nbh_indx)

# 4) Make ksyn synthetic datapoints

In [ ]:
k2 = 20

In [ ]:
def perturb(flat_k,points_per_dp=k2):
    num_codes = flat_k.shape[1]
    feat_mx = np.full((k1*k2,num_codes),0.)
    for f in range(num_codes):
        feat_mean = np.mean(flat_k[:,f])
        feat_std = np.std(flat_k[:,f])
        feat_mx[:,f]=np.random.normal(feat_mean,feat_std,k1*k2)
    feat_mx[feat_mx<0.]=0.
    feat_mx[feat_mx>1.]=1.
    return feat_mx

In [ ]:
wup_syn_flat = perturb(wup_nbh_flat)
jac_syn_flat = perturb(jac_nbh_flat)

# 5) Unflatten

In [ ]:
max_past = max([len(x) for x in mimic])
past_mean = np.mean([len(x) for x in mimic])
past_std = np.std([len(x) for x in mimic])

In [ ]:
def unflat_cloud(cloud,code_list):
    res = []
    for dp in cloud: #iterate over synthetic datapoints
        dp_mx = np.full((len(code_list),max_past),0) #codes * visits
        for col,temp_val in enumerate(dp):
            check = .5
            for visit_index in range(max_past-1,-1,-1):
                if temp_val>=check:
                    dp_mx[col,visit_index]=1
                    temp_val-=check
                check/=2
            #dp_mx matrix filled with values
        raw_seq = [[code_list[c] for c in range(dp_mx.shape[0]) if dp_mx[c,v]==1] for v in range(dp_mx.shape[1])]#
        num_visit = max(1,int(np.ceil(np.random.normal(past_mean,past_std))))
        res.append(raw_seq[-num_visit:])
    return res

In [ ]:
wup_syn_seq = unflat_cloud(wup_syn_flat,wup_codes)
jac_syn_seq = unflat_cloud(jac_syn_flat,jac_codes)

# 6) [toy] caching

In [ ]:
mega_dict = {}
mega_dict[(i2e_index,'jac')] = (jac_nbh_indx,jac_nbh_flat,jac_syn_flat,jac_syn_seq)
mega_dict[(i2e_index,'wup')] = (wup_nbh_indx,wup_nbh_flat,wup_syn_flat,wup_syn_seq)
pickle.dump(mega_dict,open('mimic_cache.pkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
test = pickle.load(open('mimic_cache.pkl','rb'))
print(test.keys())

# 7) MEGALOOP

In [ ]:
len(mimic)

In [ ]:
jac_dict = {}
wup_dict = {}

for indx in range(len(mimic)):
    if indx%100==0:
        print(indx,time.ctime())
    try:
        i2e = mimic[indx]
        wup_nbh_indx = [el[0] for el in get_closest_datapoints_indexes(indx,p2p_wuplus,k1)] 
        jac_nbh_indx = [el[0] for el in get_closest_datapoints_indexes(indx,p2p_jaccard,k1)] 
        wup_nbh_flat,wup_codes = flat_cloud(wup_nbh_indx)
        jac_nbh_flat,jac_codes = flat_cloud(jac_nbh_indx)
        wup_syn_flat = perturb(wup_nbh_flat)
        jac_syn_flat = perturb(jac_nbh_flat)
        wup_syn_seq = unflat_cloud(wup_syn_flat,wup_codes)#
        jac_syn_seq = unflat_cloud(jac_syn_flat,jac_codes)  
        wup_dict[indx] = (wup_nbh_indx,wup_nbh_flat,wup_syn_flat,wup_syn_seq)
        jac_dict[indx] = (jac_nbh_indx,jac_nbh_flat,jac_syn_flat,jac_syn_seq)
    except Exception as ex:
        print('ERROR',indx)

In [ ]:
missed = []
for i in range(len(mimic)):
    try:
        wup_dict[i]
        jac_dict[i]
    except:
        missed.append(i)

In [ ]:
np.array(missed)

In [ ]:
pickle.dump(jac_dict,open('../data/jac_cache_2.pkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)    

In [ ]:
pickle.dump(wup_dict,open('../data/wup_cache_2.pkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)    

In [ ]:
len(list(pickle.load(open('../data/jac_cache_2.pkl','rb')).keys()))

In [ ]:
len(list(pickle.load(open('../data/wup_cache_2.pkl','rb')).keys()))